# Effective Bounds: A<sup>eff</sup>, B<sup>eff</sup> etc.

Implementation of $A^{eff}$, $B^{eff}$ using the technique illustrated in **Effective Bounds: A', B'**. As before, the forumlae below are cut and pasted into markdown cells of this document from LaTeX provided by MathJax on the [web page](http://michaelnielsen.org/polymath1/index.php?title=Polymath15_test_problem).

$$A^{eff} := \frac{1}{8} \exp( \frac{t}{4} \alpha_1(\frac{1-y+ix}{2})^2 ) H_{0,1}(\frac{1-y+ix}{2}) \sum_{n=1}^N \frac{1}{n^{\frac{1-y+ix}{2} + \frac{t \alpha_1(\frac{1-y+ix}{2})}{2} - \frac{t}{4} \log n}}$$

$$B^{eff} := \frac{1}{8} \exp( \frac{t}{4} \overline{\alpha_1(\frac{1+y+ix}{2})}^2 ) \overline{H_{0,1}(\frac{1+y+ix}{2})} \sum_{n=1}^N \frac{1}{n^{\frac{1+y-ix}{2} + \frac{t \overline{\alpha_1(\frac{1+y+ix}{2})}}{2} - \frac{t}{4} \log n}}$$

(Note: The following probably should read $B_0^{eff}$ rather than $B^{eff}$ as it is just the previous formula with N=1.)

$$B^{eff} := \frac{1}{8} \exp( \frac{t}{4} \overline{\alpha_1(\frac{1+y+ix}{2})}^2 ) \overline{H_{0,1}(\frac{1+y+ix}{2})}$$

$$H_{0,1}(s) := \frac{s (s-1)}{2} \pi^{-s/2} \sqrt{2\pi} \exp( (\frac{s}{2} - \frac{1}{2}) \log \frac{s}{2} - \frac{s}{2} )$$

$$\alpha_1(s) := \frac{1}{2s} + \frac{1}{s-1} + \frac{1}{2} \log \frac{s}{2\pi}$$

$$N := \lfloor \sqrt{ \frac{T'}{2\pi}} \rfloor$$

$$T' := \frac{x}{2} - \frac{\pi t}{8}.$$

Also recall that if $z= x+iy$, then $s=\frac{1+zi}{2}=\frac{1-y+ix}{2}$. Note that the expression $\frac{1+y+ix}{2}=\frac{1+\bar{z}i}{2}$ occurs in the formula for $B^{eff}.$ The expression $\frac{1+y-ix}{2}$ which occurs in the denominator of $B^{eff}$'s summand does *not* appear to be a typo. It is consistent with the Python implemetation, [Ht_Effective(z, t)](https://github.com/km-git-acc/dbn_upper_bound/blob/d1cc87111711f7d1e826b9d253c1453a01489561/dbn_upper_bound/python/mputility.py#L386), specifically the definition of `s1` as $\frac{1-y+ix}{2}$ and the use of `1-s1` or $\frac{1+y-ix}{2}$ in the associated summation loop. The formulae above thus appear correct.

The last two formulae may be combined into as follows. Since $s = \frac{1-y+ix}{2}$, $\frac{x}{2}= imag(s)$, and $T' = imag(s)- \frac{\pi t}{8}$. Hence,

In [3]:
using DBNUpperBound
using DBNUpperBound.Asymptotics

Nseff(s,t) = floor(Int, √((imag(s) - π*t/8)/(2*π))) # Admissable function definition syntax in Julia

Nseff (generic function with 1 method)

Implementing $\alpha_1$ as $\alpha$eff:

$$\alpha_1(s) := \frac{1}{2s} + \frac{1}{s-1} + \frac{1}{2} \log \frac{s}{2\pi}$$

In [4]:
αeff(s) = 1/(2*s) + 1/(s-1) + 1/2*log(s/(2*π))

αeff (generic function with 1 method)

Continuing with $H_{0,1}$

$$H_{0,1}(s) := \frac{s (s-1)}{2} \pi^{-s/2} \sqrt{2\pi} \exp( (\frac{s}{2} - \frac{1}{2}) \log \frac{s}{2} - \frac{s}{2} )$$

In [5]:
H01(s) = (s*(s-1)/2)*√(2*π)*bigexp(-s/2*log(π) + (s/2-1/2)*log(s/2) - s/2)

H01 (generic function with 1 method)

The denominators of the summands in $A^{eff}$ and $B^{eff}$ are

$$ n^{\frac{1-y+ix}{2} + \frac{t \alpha_1(\frac{1-y+ix}{2})}{2} - \frac{t}{4} \log n}$$

for $A^{eff}$ and

$$n^{\frac{1+y-ix}{2} + \frac{t \overline{\alpha_1(\frac{1+y+ix}{2})}}{2} - \frac{t}{4} \log n}$$

for $B^{eff}$.